In [1]:
import numpy as np
import pandas as pd

In [3]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [4]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_17607/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,

In [5]:
adv_roles = True

## Imports (for potential use)

In [6]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [7]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [8]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [9]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[14:15:38] Stdout logging level is INFO2.
[14:15:38] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[14:15:38] Task: multilabel

[14:15:38] Start automl preset with listed constraints:
[14:15:38] - time: 3600.00 seconds
[14:15:38] - CPU: 4 cores
[14:15:38] - memory: 16 GB

[14:15:38] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[14:15:42] Feats was rejected during automatic roles guess: []
[14:15:42] Layer 1 train process start. Time left 3596.08 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[14:15:42] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:15:42] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:15:45] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:15:48] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[14:15:50] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.349868186133639
[14:15:50] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:15:50] Time left 3587.93 secs



...

[14:16:06] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[14:16:14] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[14:16:22] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.3226786985020595
[14:16:22] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[14:16:22] Time left 3555.61 secs

[14:16:22] Layer 1 training completed.

[14:16:22] Blending: optimization starts with equal weights and score -7.316020705556811
[14:16:22] Blending: iteration 0: score = -7.315776488114107, weights = [0.58425945 0.41574055]
[14:16:23] Blending: iteration 1: score = -7.315776488114107, weights = [0.58425945 0.41574055]
[14:16:23] Blending: no score update. Terminated

[14:16:23] Automl preset training completed in 44.81 seconds

[14:16:23] Model description:
Final prediction for new objects (level 0) = 
	 0.58426 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.41574 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [10]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb
[14:16:25] CatBoost uses as obj. MultiCrossEntropy.


In [11]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['xgb', 'linear_l2']]}
)

In [12]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[14:16:25] Stdout logging level is INFO2.
[14:16:25] Task: multilabel

[14:16:25] Start automl preset with listed constraints:
[14:16:25] - time: 3600.00 seconds
[14:16:25] - CPU: 1 cores
[14:16:25] - memory: 16 GB

[14:16:25] Train data shape: (14644, 22)
[14:16:26] Feats was rejected during automatic roles guess: []
[14:16:26] Layer 1 train process start. Time left 3599.26 secs
[14:16:27] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:16:27] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:16:34] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:16:39] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:16:46] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.377234381407589
[14:16:46] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:16:46] Time left 3579.77 secs

[14:16:48] Stdout logging level is INFO.
[14:16:48] GDBT train starts. Max iter 3000, early stopping roun

In [13]:
gpu_inf = automl_gpu.predict(data)

In [14]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[14:18:30] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[14:18:31] CatBoost uses as obj. MultiCrossEntropy.


In [15]:
cpu_inf = automl_gpu.predict(data)

In [26]:
from joblib import dump, load
import time
pickle_file = './mgpu.joblib'

In [28]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.398s


In [29]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 0.300s


In [30]:
disk_pred = model_inf.predict(data)

In [25]:
disk_pred.data.T

array([[0.10221376, 0.10451977, 0.08377431, ..., 0.18453708, 0.05863962,
        0.08744805],
       [0.00290644, 0.00713337, 0.00315106, ..., 0.00285614, 0.00371901,
        0.00250194],
       [0.02788687, 0.04403486, 0.03180921, ..., 0.12996843, 0.03031445,
        0.02972678],
       ...,
       [0.02208696, 0.03384437, 0.02286838, ..., 0.02115676, 0.02428398,
        0.03041539],
       [0.00616315, 0.01186335, 0.00667042, ..., 0.03494588, 0.00519514,
        0.00700117],
       [0.00050123, 0.00065707, 0.0005046 , ..., 0.0007624 , 0.00045758,
        0.0005862 ]], dtype=float32)

In [31]:
disk_pred.data.T

array([[0.09704998, 0.09807919, 0.07827161, ..., 0.17803985, 0.05872025,
        0.08248717],
       [0.00262182, 0.00592732, 0.00292368, ..., 0.00255388, 0.00355478,
        0.0031434 ],
       [0.02592643, 0.04273103, 0.03054169, ..., 0.12548725, 0.02884001,
        0.02651008],
       ...,
       [0.02250601, 0.03593313, 0.02242502, ..., 0.02114065, 0.02320173,
        0.03017565],
       [0.00606457, 0.01150178, 0.00683698, ..., 0.03095673, 0.00521228,
        0.00700879],
       [0.00050173, 0.00066636, 0.00050603, ..., 0.00080395, 0.00045809,
        0.00058482]], dtype=float32)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(disk_pred.data.T)

In [16]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 17607. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-12-27 14:18:43,906 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-bqb3iyus', purging
2022-12-27 14:18:43,906 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-12-27 14:18:43,906 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:35373': None}

In [17]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[14:18:44] CatBoost uses as obj. MultiCrossEntropy.
[14:18:44] Stdout logging level is INFO2.
[14:18:44] Task: multilabel

[14:18:44] Start automl preset with listed constraints:
[14:18:44] - time: 3600.00 seconds
[14:18:44] - CPU: 1 cores
[14:18:44] - memory: 16 GB

[14:18:44] Train data shape: (14644, 22)
[14:18:45] Feats was rejected during automatic roles guess: []
[14:18:45] Layer 1 train process start. Time left 3599.35 secs
[14:18:45] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:18:45] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:18:50] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:18:56] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[14:19:05] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.3768672960830965
[14:19:05] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:19:05] Time left 3579.43 secs

[14:19:05] Stdout 

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[14:19:11] task [xgboost.dask-0]:ucx://127.0.0.1:35373 got new rank 0


[14:19:45] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[14:19:45] task [xgboost.dask-0]:ucx://127.0.0.1:35373 got new rank 0


[14:20:18] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[14:20:18] task [xgboost.dask-0]:ucx://127.0.0.1:35373 got new rank 0


[14:20:51] Fitting Lvl_0_Pipe_1_Mod_0_XGB finished. score = -7.421431492538896
[14:20:51] Lvl_0_Pipe_1_Mod_0_XGB fitting and predicting completed
[14:20:51] Time left 3473.23 secs

[14:20:51] Layer 1 training completed.

[14:20:51] Blending: Optimization starts with equal weights and score -7.372348778411783
[14:20:51] Blending, iter 0: score = -7.333945528120495, weights = [0.90983003 0.09016994]
[14:20:51] Blending, iter 1: score = -7.333945528120495, weights = [0.90983003 0.09016994]
[14:20:51] No score update. Terminated
[14:20:51] Automl preset training completed in 126.95 seconds

[14:20:51] Model description:
Final prediction for new objects (level 0) = 
	 0.90983 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.09017 * (3 averaged models Lvl_0_Pipe_1_Mod_0_XGB) 



In [18]:
automl_mgpu.to_cpu()

multilabel isn`t supported in lgb
[14:20:51] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[14:20:51] CatBoost uses as obj. MultiCrossEntropy.


In [19]:
mcpu_inf = automl_mgpu.predict(data)

In [20]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

[[0.10221376 0.10451977 0.08377431 ... 0.18453708 0.05863962 0.08744805]
 [0.00290644 0.00713337 0.00315106 ... 0.00285614 0.00371901 0.00250194]
 [0.02788687 0.04403486 0.03180921 ... 0.12996843 0.03031445 0.02972678]
 ...
 [0.02208696 0.03384437 0.02286838 ... 0.02115676 0.02428398 0.03041539]
 [0.00616315 0.01186335 0.00667042 ... 0.03494588 0.00519514 0.00700117]
 [0.00050123 0.00065707 0.0005046  ... 0.0007624  0.00045758 0.0005862 ]]

[[0.08421619 0.08749928 0.07278112 ... 0.16958718 0.05770845 0.08164401]
 [0.00253699 0.00527906 0.00285997 ... 0.0023211  0.00337974 0.00222165]
 [0.02541809 0.03976582 0.02874123 ... 0.13317053 0.02973957 0.02770596]
 ...
 [0.02153561 0.03375423 0.02241501 ... 0.02133748 0.02291011 0.02957804]
 [0.00607976 0.01183458 0.00663969 ... 0.01648725 0.00517318 0.00700248]
 [0.00050123 0.00065707 0.0005046  ... 0.0007624  0.00045758 0.0005862 ]]

[[0.07701039 0.07912245 0.07914824 ... 0.17314239 0.05089302 0.09089656]
 [0.00248518 0.00605999 0.00313853 ..

In [ ]:
from joblib import dump, load
import time

pickle_file = './gpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu.client, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

In [ ]:
automl_mgpu.levels[0][0].ml_algos